In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mangoleafdb/MangoLeafBD_Without_Testset_Augmentation/Test/Powdery Mildew/IMG_20211107_125157 (Custom).jpg
/kaggle/input/mangoleafdb/MangoLeafBD_Without_Testset_Augmentation/Test/Powdery Mildew/20211109_122241 (Custom).jpg
/kaggle/input/mangoleafdb/MangoLeafBD_Without_Testset_Augmentation/Test/Powdery Mildew/20211109_121825 (Custom).jpg
/kaggle/input/mangoleafdb/MangoLeafBD_Without_Testset_Augmentation/Test/Powdery Mildew/20211109_121904 (Custom).jpg
/kaggle/input/mangoleafdb/MangoLeafBD_Without_Testset_Augmentation/Test/Powdery Mildew/IMG_20211107_125750 (Custom).jpg
/kaggle/input/mangoleafdb/MangoLeafBD_Without_Testset_Augmentation/Test/Powdery Mildew/20211109_121314 (Custom).jpg
/kaggle/input/mangoleafdb/MangoLeafBD_Without_Testset_Augmentation/Test/Powdery Mildew/20211109_121715 (Custom).jpg
/kaggle/input/mangoleafdb/MangoLeafBD_Without_Testset_Augmentation/Test/Powdery Mildew/IMG_20211107_123635 (Custom).jpg
/kaggle/input/mangoleafdb/MangoLeafBD_Without_Testset_Augmen

In [1]:
!pip install timm torch torchvision


In [2]:
import os
import torch
import torchvision
import timm
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split


In [3]:
!pip install kaggle



In [4]:
import os
print(os.listdir("/kaggle/input/"))


['mangodataset']


In [5]:
import os
print(os.listdir("/kaggle/input/mangodataset/"))  # Check files inside the folder


['mango_dataset_agumentation']


In [6]:
import os

# Check working directory
print(os.listdir("/kaggle/working/"))

# Check extracted dataset folder
print(os.listdir("/kaggle/input/mangodataset/mango_dataset_agumentation/"))  


['.virtual_documents']
['Test', 'Train']


In [9]:
import os

# Correct dataset path
dataset_path = "/kaggle/input/mangodataset/mango_dataset_agumentation/"

# Check if Train and Test folders exist
print("Contents of dataset:", os.listdir(dataset_path))  


Contents of dataset: ['Test', 'Train']


In [10]:
train_dir = os.path.join(dataset_path, "Train")  
test_dir = os.path.join(dataset_path, "Test")

print("Training Data:", os.listdir(train_dir))  
print("Test Data:", os.listdir(test_dir))  


Training Data: ['Powdery Mildew', 'Cutting Weevil', 'Anthracnose', 'Bacterial Canker', 'Sooty Mould', 'Gall Midge', 'Healthy', 'Die Back']
Test Data: ['Powdery Mildew', 'Cutting Weevil', 'Anthracnose', 'Bacterial Canker', 'Sooty Mould', 'Gall Midge', 'Healthy', 'Die Back']


In [11]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Check if this matches training
])


In [12]:
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

num_classes = len(train_dataset.classes)  # Get the number of classes
print(f"Number of classes: {num_classes}")


Number of classes: 8


In [13]:
class ViTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ViTClassifier, self).__init__()
        self.model = timm.create_model("vit_base_patch16_224", pretrained=True)
        self.model.head = nn.Linear(self.model.head.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ViTClassifier(num_classes=num_classes).to(device)


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)


In [14]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# Define dataset paths
train_dir = os.path.join(dataset_path, "Train")  
test_dir = os.path.join(dataset_path, "Test")

# Define image transformations
from torchvision import transforms
import torch.nn as nn

class CustomViT(nn.Module):
    def __init__(self, original_vit):
        super().__init__()
        self.vit = original_vit
        self.dropout = nn.Dropout(0.3)  # Apply dropout
        self.fc = nn.Linear(1000, num_classes)  # Adjust last layer

    def forward(self, x):
        x = self.vit(x)
        x = self.dropout(x)  
        return self.fc(x)

transform = transforms.Compose([
    
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1,1]


])

# Load datasets
train_dataset = ImageFolder(root=train_dir, transform=transform)
test_dataset = ImageFolder(root=test_dir, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print("Train DataLoader initialized successfully!")


Train DataLoader initialized successfully!


In [14]:
# Directory to save model checkpoints
checkpoint_dir = "/kaggle/working/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)  # Create directory if it doesn't exist

In [15]:
import os

checkpoint_dir = "/kaggle/working/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)  # Create directory if it doesn't exist

print("Checkpoint Directory Exists:", os.path.exists(checkpoint_dir))
print("Files in /kaggle/working/:", os.listdir("/kaggle/working/"))


Checkpoint Directory Exists: True
Files in /kaggle/working/: ['.virtual_documents', 'checkpoints']


In [15]:
import os

checkpoint_dir = "/kaggle/working/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)  # Create directory if it doesn't exist

print("Checkpoint Directory Exists:", os.path.exists(checkpoint_dir))
print("Files in /kaggle/working/:", os.listdir("/kaggle/working/"))

Checkpoint Directory Exists: True
Files in /kaggle/working/: ['checkpoints', '.virtual_documents']


In [14]:
import os
import zipfile
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [17]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(train_dataset.classes)

num_classes = len(train_dataset.classes)

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=num_classes,
    ignore_mismatched_sizes=True  # ✅ Fix size mismatch
)
model.to(device)


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([8]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([8, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [19]:
Criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

In [24]:
import torch
import torch.nn as nn

def train(model, train_loader, criterion, optimizer, device):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # Reset gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs.logits, labels)  # Compute loss

        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"🟢 Training Loss: {avg_loss:.4f}")
    
    return avg_loss


In [108]:
def save_checkpoint(epoch, model, optimizer, loss):
    """Save model checkpoint every 5 epochs."""
    checkpoint_path = f"/kaggle/working/vit_mango_epoch{epoch}.pth"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, checkpoint_path)
    print(f"✅ Checkpoint saved at: {checkpoint_path}")

# Training loop
num_epochs = 50
for epoch in range(1, num_epochs + 1):
    train_loss = train(model, train_loader, criterion, optimizer, device)  # Replace with your training function

    # Save a checkpoint every 5 epochs
    if epoch % 5 == 0:
        save_checkpoint(epoch, model, optimizer, train_loss)



🟢 Training Loss: 0.0180
🟢 Training Loss: 0.0096
🟢 Training Loss: 0.0136
🟢 Training Loss: 0.0151
🟢 Training Loss: 0.0147
✅ Checkpoint saved at: /kaggle/working/vit_mango_epoch5.pth
🟢 Training Loss: 0.0122
🟢 Training Loss: 0.0176
🟢 Training Loss: 0.0208
🟢 Training Loss: 0.0116
🟢 Training Loss: 0.0179
✅ Checkpoint saved at: /kaggle/working/vit_mango_epoch10.pth
🟢 Training Loss: 0.0102
🟢 Training Loss: 0.0151
🟢 Training Loss: 0.0125
🟢 Training Loss: 0.0154
🟢 Training Loss: 0.0164
✅ Checkpoint saved at: /kaggle/working/vit_mango_epoch15.pth
🟢 Training Loss: 0.0145
🟢 Training Loss: 0.0143
🟢 Training Loss: 0.0164
🟢 Training Loss: 0.0214
🟢 Training Loss: 0.0131
✅ Checkpoint saved at: /kaggle/working/vit_mango_epoch20.pth
🟢 Training Loss: 0.0123
🟢 Training Loss: 0.0157
🟢 Training Loss: 0.0083
🟢 Training Loss: 0.0124
🟢 Training Loss: 0.0151
✅ Checkpoint saved at: /kaggle/working/vit_mango_epoch25.pth
🟢 Training Loss: 0.0163
🟢 Training Loss: 0.0159
🟢 Training Loss: 0.0254
🟢 Training Loss: 0.0116


In [39]:
# Save checkpoint only every 5 epochs
if (epoch + 1) % 5 == 0:
    save_checkpoint(epoch + 1, model, optimizer, avg_loss)


In [40]:
def save_checkpoint(epoch, model, optimizer, loss):
    checkpoint_path = "/kaggle/working/vit_mango_latest.pth"  # Always overwrite
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }, checkpoint_path)
    print(f"Checkpoint saved at Epoch {epoch} ✅")


In [19]:
!df -h


Filesystem                                                              Size  Used Avail Use% Mounted on
overlay                                                                 7.9T  6.1T  1.9T  77% /
tmpfs                                                                    64M     0   64M   0% /dev
shm                                                                      14G  4.0K   14G   1% /dev/shm
/dev/sda1                                                               122G  5.1G  117G   5% /opt/bin
/dev/loop1                                                               20G   76K   20G   1% /kaggle/lib
192.168.7.2:/data/kagglesdsdata/datasets/7018670/11235209/d8v7dqczla6l   78T   57T   22T  73% /kaggle/input/mangoleafdb
/dev/mapper/snap                                                        7.9T  6.1T  1.9T  77% /etc/hosts
tmpfs                                                                    16G     0   16G   0% /proc/acpi
tmpfs                                                     

In [20]:
!rm -rf /kaggle/working/*.pth  # Delete all old checkpoints
!rm -rf /kaggle/working/__pycache__/  # Remove Python cache files


In [67]:
import torch

def save_checkpoint(epoch, model, optimizer, loss):
    checkpoint_path = f"/kaggle/working/vit_mango_epoch{epoch}.pth"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, checkpoint_path)
    print(f"✅ Checkpoint saved at: {checkpoint_path}")


In [109]:
from IPython.display import FileLink

# Change the filename to your last saved checkpoint
checkpoint_filename = "vit_mango_epoch50.pth"
checkpoint_path = f"/kaggle/working/{checkpoint_filename}"

# Generate a download link
FileLink(checkpoint_path)


/kaggle/working/vit_mango_epoch50.pth

In [110]:
!zip -r /kaggle/working/model_checkpoints.zip /kaggle/working/vit_mango_epoch*.pth


  adding: kaggle/working/vit_mango_epoch10.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch15.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch20.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch25.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch30.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch35.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch40.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch45.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch50.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch5.pth (deflated 8%)


In [111]:
from IPython.display import FileLink

FileLink("/kaggle/working/model_checkpoints.zip")


/kaggle/working/model_checkpoints.zip

In [21]:
!ls -lh /kaggle/working/


total 4.0K
drwxr-xr-x 2 root root 4.0K Apr  1 09:46 checkpoints


In [95]:
!ls -lh /kaggle/working/checkpoints

total 19G
-rw-r--r-- 1 root root 8.9G Mar 27 08:54 model_checkpoints.zip
-rw-r--r-- 1 root root 983M Mar 27 07:43 vit_mango_epoch10.pth
-rw-r--r-- 1 root root 983M Mar 27 07:50 vit_mango_epoch15.pth
-rw-r--r-- 1 root root 983M Mar 27 07:56 vit_mango_epoch20.pth
-rw-r--r-- 1 root root 983M Mar 27 08:03 vit_mango_epoch25.pth
-rw-r--r-- 1 root root 983M Mar 27 08:09 vit_mango_epoch30.pth
-rw-r--r-- 1 root root 983M Mar 27 08:16 vit_mango_epoch35.pth
-rw-r--r-- 1 root root 983M Mar 27 08:22 vit_mango_epoch40.pth
-rw-r--r-- 1 root root 983M Mar 27 08:29 vit_mango_epoch45.pth
-rw-r--r-- 1 root root 983M Mar 27 08:35 vit_mango_epoch50.pth
-rw-r--r-- 1 root root 983M Mar 27 07:37 vit_mango_epoch5.pth


In [25]:
!pip install kaggle

In [81]:
!kaggle datasets init -p /kaggle/working/

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [26]:
import IPython
from IPython.display import Javascript

def download(file_path):
    file_name = file_path.split('/')[-1]
    jscode = f"""
    var link = document.createElement('a');
    link.href = '{file_path}';
    link.download = '{file_name}';
    document.body.appendChild(link);
    link.click();
    document.body.removeChild(link);
    """
    IPython.display.display(Javascript(jscode))

# Call the function with the path to the ZIP file
download('/kaggle/working/checkpoints/model_checkpoints.zip')


<IPython.core.display.Javascript object>

In [112]:
from IPython.display import FileLink
FileLink('/kaggle/working/model_checkpoints.zip')

/kaggle/working/model_checkpoints.zip

In [1]:
import torch
import torch.nn as nn
from sklearn.metrics import precision_score

def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0

    all_preds = []
    all_labels = []

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        # Assuming outputs is an object with logits (as in Hugging Face ViT)
        loss = criterion(outputs.logits, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Collect predictions for precision
        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    avg_loss = running_loss / len(train_loader)

    # Precision calculation
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"🟢 Training Loss: {avg_loss:.4f} | 🎯 Precision: {precision:.4f}")

    return avg_loss, precision
for epoch in range(1, num_epochs + 1):
    train_loss, train_precision = train(model, train_loader, criterion, optimizer, device)
    
    print(f"📘 Epoch [{epoch}/{num_epochs}] - Loss: {train_loss:.4f}, Precision: {train_precision:.4f}", flush=True)

NameError: name 'num_epochs' is not defined

In [20]:
import torch
import torch.nn as nn
from sklearn.metrics import precision_score

def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        # Assuming outputs is an object with logits (as in Hugging Face ViT)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Collect predictions for precision calculation
        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    avg_loss = running_loss / len(train_loader)
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"🟢 Training Loss: {avg_loss:.4f} | 🎯 Precision: {precision:.4f}")
    return avg_loss, precision

# Training configuration
num_epochs = 50
model_save_path = "/kaggle/working/mango_disease.pth"

# Training loop
for epoch in range(1, num_epochs + 1):
    train_loss, train_precision = train(model, train_loader, Criterion, optimizer, device)
    print(f"📘 Epoch [{epoch}/{num_epochs}] - Loss: {train_loss:.4f}, Precision: {train_precision:.4f}", flush=True)

# Save the trained model
torch.save(model.state_dict(), model_save_path)
print(f"✅ Model saved at: {model_save_path}")

# Load the model for future use
model.load_state_dict(torch.load(model_save_path))
model.to(device)
model.eval()
print("✅ Model loaded successfully!")


🟢 Training Loss: 0.0853 | 🎯 Precision: 0.9797
📘 Epoch [1/50] - Loss: 0.0853, Precision: 0.9797
🟢 Training Loss: 0.0345 | 🎯 Precision: 0.9886
📘 Epoch [2/50] - Loss: 0.0345, Precision: 0.9886
🟢 Training Loss: 0.0010 | 🎯 Precision: 1.0000
📘 Epoch [3/50] - Loss: 0.0010, Precision: 1.0000
🟢 Training Loss: 0.0006 | 🎯 Precision: 1.0000
📘 Epoch [4/50] - Loss: 0.0006, Precision: 1.0000
🟢 Training Loss: 0.0003 | 🎯 Precision: 1.0000
📘 Epoch [5/50] - Loss: 0.0003, Precision: 1.0000
🟢 Training Loss: 0.0003 | 🎯 Precision: 1.0000
📘 Epoch [6/50] - Loss: 0.0003, Precision: 1.0000
🟢 Training Loss: 0.0002 | 🎯 Precision: 1.0000
📘 Epoch [7/50] - Loss: 0.0002, Precision: 1.0000


KeyboardInterrupt: 

In [28]:
model.load_state_dict(torch.load(model_save_path, weights_only=True))


<All keys matched successfully>

In [30]:
torch.save(model.state_dict(), "model.pth")


In [26]:
import torch
from PIL import Image

def predict_image(image_path, model, transform, class_names):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension
    
    model.eval()
    with torch.no_grad():
        output = model(image)
        preds = torch.argmax(output.logits, dim=1).item()
    
    print(f"Predicted Class: {class_names[preds]}")

# Example usage:
class_names = ['Anthracnose', 'Bacterial Canker', 'Cutting Weevil', 
           'Die Back', 'Gall Midge', 'Healthy', 'Powdery Mildew', 'Sooty Mould']
predict_image("test_leaf.jpg", model, transform, class_names)


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/test_leaf.jpg'

In [25]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the transformation for training and testing data
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the training and test datasets
train_dataset = datasets.ImageFolder(root='/kaggle/input/mangodataset/mango_dataset_agumentation/Train/', transform=transform_train)
test_dataset = datasets.ImageFolder(root='/kaggle/input/mangodataset/mango_dataset_agumentation/Test/', transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load Vision Transformer (ViT) model pre-trained on ImageNet
model = torchvision.models.vision_transformer.vit_b_16(weights=torchvision.models.ViT_B_16_Weights.IMAGENET1K_V1)

# Print model structure to inspect the last layer
print(model)

# Modify the final classification layer to match the number of classes in the mango dataset (adjust as needed)
num_classes = len(train_dataset.classes)

# Access the final layer in the model's head and modify it
in_features = model.heads[0].in_features  # Get the input features of the last linear layer
model.heads[0] = nn.Linear(in_features, num_classes)  # Replace the last linear layer with one having the correct output size

# Move the model to the appropriate device
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    
    # Iterate over the training data
    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        images, labels = images.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        
        # Get predictions
        _, preds = torch.max(outputs, 1)
        correct_preds += torch.sum(preds == labels).item()
        total_preds += labels.size(0)
    
    # Print epoch statistics
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = correct_preds / total_preds * 100
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

# Save the trained model
model_save_path = "/kaggle/working/mango_leaf_disease.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

# Evaluation loop on test data
model.eval()
correct_preds = 0
total_preds = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct_preds += torch.sum(preds == labels).item()
        total_preds += labels.size(0)

test_accuracy = correct_preds / total_preds * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")


VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

Epoch 1/50: 100%|██████████| 97/97 [01:09<00:00,  1.40it/s]


Epoch [1/50] Loss: 0.1609, Accuracy: 95.87%


Epoch 2/50: 100%|██████████| 97/97 [01:08<00:00,  1.41it/s]


Epoch [2/50] Loss: 0.0117, Accuracy: 99.64%


...

Epoch [48/50] Loss: 0.0000, Accuracy: 100.00%


Epoch 49/50: 100%|██████████| 97/97 [01:08<00:00,  1.41it/s]


Epoch [49/50] Loss: 0.0000, Accuracy: 100.00%


Epoch 50/50: 100%|██████████| 97/97 [01:08<00:00,  1.41it/s]


Epoch [50/50] Loss: 0.0000, Accuracy: 100.00%
Model saved to /kaggle/working/mango_leaf_disease.pth
Test Accuracy: 98.87%


In [ ]:
/kaggle/working/mango_leaf_disease.pth
/kaggle/input/mangodataset/mango_dataset_agumentation/Train/

In [27]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [28]:
print(train_dataset.classes)  # Check the order of class labels


['Anthracnose', 'Bacterial Canker', 'Cutting Weevil', 'Die Back', 'Gall Midge', 'Healthy', 'Powdery Mildew', 'Sooty Mould']
